In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')) # or the path to your source code
sys.path.insert(0, module_path)
os.chdir('..')

In [3]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

In [4]:
from data.data import LoadData

dataset = LoadData("SBM_CLUSTER")

Using backend: pytorch


[I] Loading dataset SBM_CLUSTER...
train, test, val sizes : 10000 1000 1000
[I] Finished loading.
[I] Data load time: 7.7512s


In [5]:
import torch.nn as nn
from torch.utils.data import DataLoader
import torch
import numpy as np
from sklearn import svm
import dgl

In [6]:
train_loader = DataLoader(dataset.train, batch_size=len(dataset.train), shuffle=True, collate_fn=dataset.collate)
test_loader = DataLoader(dataset.test, batch_size=len(dataset.test), shuffle=False, collate_fn=dataset.collate)
val_loader = DataLoader(dataset.val, batch_size=len(dataset.val), shuffle=False, collate_fn=dataset.collate)

In [7]:
import pandas as pd

for name, loader in zip(['train', 'test', 'val'], [train_loader, test_loader, val_loader]):
    edge_list_data = []
    for batch_graphs, batch_targets in loader:
        graphs = dgl.unbatch(batch_graphs)
        for g in graphs:
            src, dst = g.edges()
            edge_index = [src.tolist(), dst.tolist()]
            num_nodes = g.num_nodes()
            edge_list_data.append({'edge_index': edge_index, 'num_nodes': num_nodes})
    edge_list_df = pd.DataFrame(edge_list_data)
    edge_list_df.to_csv(f'supp_data/temp/cluster_graphs_{name}.csv', index=False)